<a href="https://colab.research.google.com/github/tannisthamaiti/Market-Analytics/blob/main/Copy_of_Transformer_chatbot_Answer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets torch transformers
!wget http://www.cs.cornell.edu/~cristian/data/cornell_movie_dialogs_corpus.zip
!unzip -qq cornell_movie_dialogs_corpus.zip
!rm cornell_movie_dialogs_corpus.zip
!mkdir datasets
!mv cornell\ movie-dialogs\ corpus/movie_conversations.txt ./datasets
!mv cornell\ movie-dialogs\ corpus/movie_lines.txt ./datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 29.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 49.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 78.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from collections import Counter
import json
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.utils.data
import math
import torch.nn.functional as F

## 1) Data Processing

In [3]:
# data processing
max_len = 25

def remove_punc(string):
    punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
    no_punct = ""
    for char in string:
        if char not in punctuations:
            no_punct = no_punct + char  # space is also a character
    return no_punct.lower()

In [4]:
corpus_movie_conv = './datasets/movie_conversations.txt'
corpus_movie_lines = './datasets/movie_lines.txt'
with open(corpus_movie_conv, 'r', encoding='iso-8859-1') as c:
    conv = c.readlines()
with open(corpus_movie_lines, 'r', encoding='iso-8859-1') as l:
    lines = l.readlines()

## Check first 5 lines

In [5]:
line_iter = iter(lines)
lines_dic = {}

for _ in range(5):
    try:
        line = next(line_iter)
        print(line)

    except StopIteration:
        break

L1045 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ They do not!

L1044 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ They do to!

L985 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ I hope so.

L984 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ She okay?

L925 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Let's go.



In [6]:
conv_iter = iter(conv)
conv_dic = {}

for _ in range(5):
    try:
        line = next(conv_iter)
        print(line)

    except StopIteration:
        break

u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L194', 'L195', 'L196', 'L197']

u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L198', 'L199']

u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L200', 'L201', 'L202', 'L203']

u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L204', 'L205', 'L206']

u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L207', 'L208']



## Extract for a single line

In [12]:
lines_dic_example = {}
line_example = lines[1]
objects = line.split(" +++$+++ ")
print(objects)
lines_dic_example[objects[0]] = objects[-1]
print(lines_dic_example)

['L666256', 'u9034', 'm616', 'VEREKER', "Colonel Durnford... William Vereker. I hear you 've been seeking Officers?\n"]
{'L666256': "Colonel Durnford... William Vereker. I hear you 've been seeking Officers?\n"}


## Extract for a single conversation

In [19]:
conv_dic_example = {}
con_example = conv[1]
ids = eval(con_example.split(" +++$+++ ")[-1])
print(ids)
for i in range(len(ids)):
  qa_pairs = []
  if i == len(ids) - 1:
    break
  first = remove_punc(lines_dic[ids[i]].strip())
  second = remove_punc(lines_dic[ids[i+1]].strip())
  print("First conversation is: ",first)
  print("Second conversation is: ",second)
  qa_pairs.append(first.split()[:max_len])
  qa_pairs.append(second.split()[:max_len])
  print(qa_pairs)


['L198', 'L199']
First conversation is:  youre asking me out  thats so cute whats your name again
Second conversation is:  forget it
[['youre', 'asking', 'me', 'out', 'thats', 'so', 'cute', 'whats', 'your', 'name', 'again'], ['forget', 'it']]


## Extarct for entire dataset

In [7]:
# extract text
lines_dic = {}
for line in lines:
    objects = line.split(" +++$+++ ")
    lines_dic[objects[0]] = objects[-1]

# generate question answer pairs
pairs = []
for con in conv:
    ids = eval(con.split(" +++$+++ ")[-1])
    for i in range(len(ids)):
        qa_pairs = []

        if i == len(ids) - 1:
            break
        first = remove_punc(lines_dic[ids[i]].strip())
        second = remove_punc(lines_dic[ids[i+1]].strip())
        qa_pairs.append(first.split()[:max_len])
        qa_pairs.append(second.split()[:max_len])
        pairs.append(qa_pairs)


[['i', 'really', 'really', 'really', 'wanna', 'go', 'but', 'i', 'cant', 'not', 'unless', 'my', 'sister', 'goes'], ['im', 'workin', 'on', 'it', 'but', 'she', 'doesnt', 'seem', 'to', 'be', 'goin', 'for', 'him']]


## Create Word Embedding

create a `word_freq`counter, Counter() is a special dictionary from the collections module that automatically counts how many times each item (like a word) appears.

In [ ]:
# word map
min_word_freq = 5

word_freq = Counter()
for pair in pairs:
    word_freq.update(pair[0])
    word_freq.update(pair[1])

In [8]:


words = [w for w in word_freq.keys() if word_freq[w] > min_word_freq]
word_map = {k: v + 1 for v, k in enumerate(words)}
word_map['<unk>'] = len(word_map) + 1
word_map['<start>'] = len(word_map) + 1
word_map['<end>'] = len(word_map) + 1
word_map['<pad>'] = 0

print("Total words are: {}".format(len(word_map)))

# encode sentences based on word map
def encode_question(words, word_map):
    enc_c = [word_map.get(word, word_map['<unk>']) for word in words] + [word_map['<pad>']] * (max_len - len(words))
    return enc_c

def encode_reply(words, word_map):
    enc_c = [word_map['<start>']] + [word_map.get(word, word_map['<unk>']) for word in words] + \
        [word_map['<end>']] + [word_map['<pad>']] * (max_len - len(words))
    return enc_c

pairs_encoded = []
for pair in pairs:
    qus = encode_question(pair[0], word_map)
    ans = encode_reply(pair[1], word_map)
    pairs_encoded.append([qus, ans])


Total words are: 18243


In [9]:
# dataset and dataloader
class Dataset(Dataset):

    def __init__(self, pairs):

        self.pairs = pairs
        self.dataset_size = len(self.pairs)

    def __getitem__(self, i):
        question = torch.LongTensor(self.pairs[i][0])
        reply = torch.LongTensor(self.pairs[i][1])
        return question, reply

    def __len__(self):
        return self.dataset_size

train_loader = DataLoader(Dataset(pairs_encoded), batch_size=32, shuffle=True, pin_memory=True)
question, reply = next(iter(train_loader))
print("Question: ", question.size())
print("Answer: ", reply.size())

Question:  torch.Size([32, 25])
Answer:  torch.Size([32, 27])


In [10]:
# create mask
def create_masks(question, reply_input, reply_target, device='cpu'):

    def subsequent_mask(size):
        # (max_words, max_words)
        # binary triangle
        mask = torch.triu(torch.ones(size, size)).transpose(0, 1).type(dtype=torch.uint8)
        # (1, max_words, max_words)
        return mask.unsqueeze(0)

    # boolean(m, max_words)
    question_mask = question != 0

    # (m, 1, 1, max_words)
    question_mask = question_mask.to(device)
    question_mask = question_mask.unsqueeze(1).unsqueeze(1)

    # boolean(m, max_words)
    reply_input_mask = reply_input != 0
    # (m, 1, max_words)
    reply_input_mask = reply_input_mask.unsqueeze(1)

    # only include triangle and non-pad token
    # (m, max_words, max_words)
    reply_input_mask = reply_input_mask & subsequent_mask(reply_input.size(-1)).type_as(reply_input_mask.data)

    # (batch_size, 1, max_words, max_words)
    reply_input_mask = reply_input_mask.unsqueeze(1)

    # (batch_size, max_words)
    reply_target_mask = reply_target != 0

    return question_mask, reply_input_mask, reply_target_mask

reply_input = reply[:, :-1]
reply_target = reply[:, 1:]
print('Reply Target Size: ', reply_target.size())

# Create mask and add dimensions
question_mask, reply_input_mask, reply_target_mask = create_masks(question, reply_input, reply_target)
print('question_mask Size: ', question_mask.size())
print('reply_input_mask Size: ', reply_input_mask.size())
print('reply_target_mask Size: ', reply_target_mask.size())

Reply Target Size:  torch.Size([32, 26])
question_mask Size:  torch.Size([32, 1, 1, 25])
reply_input_mask Size:  torch.Size([32, 1, 26, 26])
reply_target_mask Size:  torch.Size([32, 26])


In [11]:
import torch
import torch.nn as nn

class EncoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, ff_dim=2048, dropout=0.1):
        super(EncoderLayer, self).__init__()

        # Multi-head self-attention
        self.self_attn = MultiHeadAttention(d_model=d_model, num_heads=num_heads)

        # Feed-forward network
        self.feed_forward = FeedForward(d_model=d_model, middle_dim=ff_dim)

        # Layer normalization
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)

        # Dropout
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)

    def forward(self, x, mask=None):
        # --- Self-Attention sublayer with residual and layer norm ---
        attn_output = self.self_attn(x, x, x)  # Self-attention
        x = self.norm1(x + self.dropout1(attn_output))

        # --- Feed-forward sublayer with residual and layer norm ---
        ff_output = self.feed_forward(x)
        x = self.norm2(x + self.dropout2(ff_output))

        return x


---------------------------------------------------------------------------------------------------------
class DecoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, ff_dim=2048, dropout=0.1):
        super(DecoderLayer, self).__init__()

        # Masked self-attention (with future mask)
        self.self_attn = MultiHeadAttention(d_model=d_model, num_heads=num_heads)

        # Encoder-decoder attention
        self.cross_attn = MultiHeadAttention(d_model=d_model, num_heads=num_heads)

        # Feed-forward network
        self.feed_forward = FeedForward(d_model=d_model, middle_dim=ff_dim)

        # Layer norms
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.norm3 = nn.LayerNorm(d_model)

        # Dropouts
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)
        self.dropout3 = nn.Dropout(dropout)

    def forward(self, x, enc_output, self_attn_mask=None, cross_attn_mask=None):
        # --- Masked Self-Attention ---
        self_attn_output = self.self_attn(x, x, x)
        x = self.norm1(x + self.dropout1(self_attn_output))

        # --- Encoder-Decoder Cross-Attention ---
        cross_attn_output = self.cross_attn(x, enc_output, enc_output)
        x = self.norm2(x + self.dropout2(cross_attn_output))

        # --- Feed-Forward Network ---
        ff_output = self.feed_forward(x)
        x = self.norm3(x + self.dropout3(ff_output))

        return x

SyntaxError: invalid syntax (<ipython-input-11-2433a106a1c5>, line 34)